In [220]:
from math import log2
from math import ceil
import pandas as pd
from random import randint

In [221]:
def check_letter(letter, alfabet):
    """Принимает символ из текста, и пустой алфавит.
    Заполняет словарь алфавит найденными символами, в качестве ключей
     и списки с частотой попадания в тексте в качестве значений"""
    if letter in alfabet.keys():
        alfabet[letter][0] += 1
    else:
        alfabet[letter] = []
        alfabet[letter].append(1)

def cut_unright_letters(res):
    """Убирает символы не соответсвующие заданию """
    res = dict(sorted(res.items()))
    del_res = list(res.keys())[1:18]
    for el in del_res:
        res.pop(el)
    del_res = list(res.keys())[-2:]
    for el in del_res:
        res.pop(el)
    res['ь'][0] += res['ъ'][0]
    res.pop('ъ')
    return res

def propability_letters(alfabet):
    """Считает вероятность попадания символа в тексте"""
    for letter in alfabet.keys():
        frequency_letter = alfabet[letter][0]
        P_letter = frequency_letter / len(article)
        alfabet[letter].append(P_letter)
    return alfabet
 

def count_letters_in_text(article):
    """Создаёт алфавит на основе переданного текста.
    Формат алфавита - словарь, где символ это ключ, а значение список, где
    первый элемент частота попадания, второй вероятность попадания"""
    alfabet = {}
    for letter in article:
        check_letter(letter, alfabet)
    #alfabet = cut_unright_letters(res=alfabet)
    alfabet = propability_letters(alfabet)
    return alfabet


def entropy_shanon(alfabet):
    """Считает энтрпию Шенонна"""
    entropy = 0
    for prob in alfabet.values():
        x = -(prob[1] * log2(prob[1]))
        entropy += x
    return entropy

def entropy_shanon_2sym(alfabet):
    """Считает энтрпию Шенонна для пар символов"""
    entropy = 0
    for prob_i in alfabet.values():
        for prob_j in alfabet.values():
            p = prob_i[1] * prob_j[1]
            x = -(p * log2(p))
            entropy += x
    return entropy

def sort_dict_value(dict1):
    """Сортирует алфавит относительно частоты попадания по убыванию"""
    sorted_values = sorted(dict1.values(), reverse=True)
    sorted_dict = {}
    for i in sorted_values:
        for k in dict1.keys():
            if dict1[k] == i:
                sorted_dict[k] = dict1[k]

    return sorted_dict

def codder(block, value):
    """Формирует код для символа алфавита или добавлет его к существующему.
    Символ кода указывается 2 аргументом"""
    for item in block:
        if len(item) == 2:
            item.append(value)
        else:
            item[2] += value

def median_propability(sym_prob):
    """Находит середину массива вероятности"""
    summ = 0
    for sym in sym_prob:
        prob = sym[1]
        summ += prob
    return summ / 2

def shannon_fano_codder(sym_prob):
    """Кодирование алфавита оптимальным методом Шеннона-Фано.
    Предварительно надо взять значения словаря и преообразовать их к списку"""
    median = median_propability(sym_prob)
    summ = 0
    index = 0
    #sym_prob = list(alfabet.valaues())
    while summ < median:
        summ += sym_prob[index][1]
        index += 1
    zero_vector = sym_prob[:index]
    one_vector = sym_prob[index:]
    codder(zero_vector, '0')
    codder(one_vector, '1')
    if len(zero_vector) > 1:
        shannon_fano_codder(zero_vector)
    if len(one_vector) > 1:
        shannon_fano_codder(one_vector)

def code_length(alfabet):
    """Вычисляет длину кодировки символа"""
    for sym in alfabet.keys():
        l = len(alfabet[sym][2])
        alfabet[sym].append(l)

#Код для 2 лабораторной работы____________________________________________________
def message_codder(text, alfabet):
    """Кодирует сообщение согласно заданому алфавиту"""
    text_code = list(text)
    for i in range(len(text_code)):
        sym = text_code[i]
        sym_code = alfabet[sym][2]
        text_code[i] = sym_code
    text_code = ''.join(text_code)
    return text_code

def inversion_noise(message, percent):
    """Генерирует случайный шум. С заданной веорятностью в процентах
    инвертирует бит сообщения"""
    percent_list = list(range(1, percent+1))
    error_message = list(message)
    for i in range(len(error_message)):
        sym = error_message[i]
        if sym == '0':
            error = randint(1, 100)
            if error in percent_list:
                error_message[i] = '1'
        elif sym == '1':
            error = randint(1, 100)
            if error in percent_list:
                error_message[i] = '0'
    error_message = ''.join(error_message)
    return error_message

def calculated_control_bits(message, count_control_bits):
    """Вычисляет контрольные биты"""
    bit = 1
    step = 2
    for i in range(count_control_bits):
        control_line = message[bit-1:]
        control_sum = []
        for j in range(0, len(control_line), step):
            if len(control_line[j:]) >= bit:
                part = control_line[j:j+bit]
                control_sum += part
            elif len(control_line[j:]) < bit:
                part = control_line[j:]
                control_sum += part
        control_sum = control_sum[1:]
        for k in range(len(control_sum)):
            control_sum[k] = int(control_sum[k])
        summ_bits = sum(control_sum)
        if summ_bits % 2 == 1:
            message[bit-1] = '1'
        bit *= 2
        step *= 2
    return message

        

def hamming_codder(message):
    """Кодирует двоичное сообщение контрольными битами"""
    coded_message = []
    num_control_bit = 1
    index_coded_message = 1
    index_old_mesage = 0
    while index_old_mesage != len(message):
        if index_coded_message == num_control_bit:
            coded_message.append('0')
            num_control_bit *= 2
            index_coded_message += 1
        else:
            coded_message.append(message[index_old_mesage])
            index_coded_message += 1
            index_old_mesage += 1
    count_control_bits = int(log2(num_control_bit))
    coded_message = calculated_control_bits(coded_message, count_control_bits)
    coded_message = ''.join(coded_message)
    return coded_message

def hamming_decodder(message):
    """Проверяет закодироаное методом Хэминга сообщения на наличие ошибок.
    Может отследить только одну ошибку"""
    message_decoded = list(message)
    n_bits = ceil(log2(len(message)))
    message_decoded = calculated_control_bits(message_decoded, n_bits)
    message_decoded = ''.join(message_decoded)
    if message_decoded == message:
        return message_decoded
    else:
        print('В сообщении ошибка!')

#def calculated_control_bits(message)

        
        
        

In [222]:
article = 'МосковскийГосударственныйТехническийУниверситет'
#article = article.lower()
article

'МосковскийГосударственныйТехническийУниверситет'

In [223]:
res = count_letters_in_text(article)
res = sort_dict_value(res)
sym_prob = list(res.values())
shannon_fano_codder(sym_prob)
code_length(res)
result_table = pd.DataFrame(data=res).T
result_table.columns = ['Частота попадания', 'Вероятность попадания', 'Кодовое слово', 'Длина кодового слова']
print('Энтропия Шеннона:', entropy_shanon(res))
print('Энтропия Шенонна пар символов:', entropy_shanon_2sym(res))
print('Средняя длина кодового слова: ', result_table['Длина кодового слова'].mean())
result_table

Энтропия Шеннона: 4.011961331166617
Энтропия Шенонна пар символов: 8.02392266233324
Средняя длина кодового слова:  4.6


,Частота попадания,Вероятность попадания,Кодовое слово,Длина кодового слова
с,6,0.12766,0000,4
и,5,0.106383,0001,4
е,5,0.106383,001,3
н,4,0.085106,0100,4
о,3,0.06383,0101,4
к,3,0.06383,011,3
в,3,0.06383,1000,4
й,3,0.06383,1001,4
т,3,0.06383,1010,4
р,2,0.042553,1011,4


In [224]:
message_codder(article, res)

'11000001010000011010110000000011000110011100010101000011001110101101110110000101010000010100010011100010011110010011110101000001111100010000011000110011111101000001100000110110000000110100011010'

In [225]:
message = message_codder(article, res)
message = inversion_noise(message, 5)
message

'11000101110000011010110000000011000110011110010111000011000010101111100110000101010000010100010010100010011110010001110101001001111100010000011000110011111101000001100000110110000001110100011010'

In [226]:
result = message_codder(article, res)
message = hamming_codder(result)
#message = inversion_noise(message, 1)
hamming_decodder(message)

'1011100100010101000011010110000100001100011001110001010100001100011101011011101100001010100000101000100111000100111100100111101001000001111100010000011000110011111101000001100000110110000000110100011010'

In [227]:
message = '0100010000111101'
print(message)
print(hamming_codder(message))

0100010000111101
100110000100001011101


In [234]:
message2 = '100110010100101011101'  #Сообщение без ошибки
print(hamming_decodder(message2))
print('----------------------------')
message2 = '100110010100101011100' #Сообщение с ошибкой
hamming_decodder(message3)

100110010100101011101
----------------------------
В сообщении ошибка!


Даже при маленькой веорятности инверсии бита информации, если передаваемое сообщение(последовательность бит) очень большая, то в нем будет с 99% вероятностью несколько ошибок. Реализованный метод Хэмимнга в данной работе может отследить только 1 ошибку в сообщении, при их большем числе может не оследить.
Для правильной передачи сообщений с помощию методов кодирования Шеннона-Фано и Хэмминга возможно такая реализация.
1. Сообщение кодируется оптимальным кодом Шеннон-Фано.
2. Закодированное сообщение дробится на несколько пакетов одинакового размера(например по байтно). В случае неполного заполненния пакета остаток заполнить нулями. Начальный и последний покет соответсвенно пометить.
3. Пакет пердать на кодирования методом Хэмминга
4. При получении пакет проверяется методом Хэмминга. В случае налиичия ошибки, она исправляется
5. Канал открывается с получением первого пакета, и закрывается полседним(для этого и нужны пометки) и закрывается с получением последнего
6. Контрольные биты изымаются из пакетов
7. Пакеты заново склеиваются
8. Сообщение декодируется обратно в человекочитаемый вид
При дроблении сообщения на части, и кодировании их по отдельности веорятность, что в пакете при передаче будет больше 1 ошибки значительно меньше, чем при предачи сообщения целиком. Однако она не нулевая поэтому не стоит делать пакеты слишком большими.